In [2]:
%pip install "pettingzoo[classic]" gymnasium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 30.1 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.5/852.5 kB 41.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 23.6 MB/s  0:00:00 eta 0:00:01
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147878 sha256=584dcf9ceab6b81be1c7ec44d0823d780a9debafc610ea4dab68ec5a6491bc0e
  Stored in directory: /Users/boatwang/Library/Caches/pip/wheels/7a/79/8e/0d6e404db9f1e82af2e40b49161d6acab485d75dfb0470ac08
  Created wheel for rlcard: filename=rlcard-1.2.0-py3-none-any.whl size=325884 sha256=7ab57e490e778adec941fcd81c86502867b2829996695de8c9082b53cbf69b20
  Stored in directory: /Users/boatwang/Library/Cache

# Rule-Based Policy

In [4]:
from pettingzoo.classic import texas_holdem_v4
from ray.rllib.env import PettingZooEnv
from ray.tune.registry import register_env

def env_creator(_config=None):
    env = texas_holdem_v4.env()
    return PettingZooEnv(env)

env_name = "texas_holdem_v4"
register_env(env_name, lambda config: env_creator(config))


In [5]:
import numpy as np
from ray.rllib.policy.policy import Policy

class RuleBasedPolicy(Policy):
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        self.action_space = action_space

    def compute_actions(
        self,
        obs_batch,
        state_batches=None,
        prev_action_batch=None,
        prev_reward_batch=None,
        info_batch=None,
        episodes=None,
        **kwargs,
    ):

        actions = []

        for obs in obs_batch:
            mask = obs[:self.action_space.n] > 0.5

            if mask[1]:
                a = 1  # Raise if allowed
            elif mask[0]:
                a = 0  # Else Call
            elif mask[3]:
                a = 3  # Else Check
            else:
                a = 2  # Else Fold

            actions.append(a)

        return np.array(actions), [], {}
